<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/Recommendation_servicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Change Directary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/ADSP Capstone/Ateema/recommendation_servicing')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pwd

# GPU Environment

In [ ]:
!nvidia-smi

Sun Aug 11 22:40:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Check if GPU is available and select it
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

# Set the default tensor type to cuda
if device.type == 'cuda':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

Using GPU: Tesla T4
Total GPU memory: 15.84 GB
Allocated GPU memory: 0.00 GB


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


# Package Install

In [ ]:
!pip install langchain-core==0.2.18 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.1 MB/s eta 0:00:00


In [ ]:
!pip install langchain==0.2.6 --quiet
!pip install langchain_community==0.2.6 --quiet
!pip install langchain-text-splitters==0.2.2 --quiet
!pip install langgraph --quiet
!pip install httpx --quiet
!pip install tiktoken --quiet
!pip install gpt4all==2.6.0 --quiet
!pip install chromadb --quiet
!pip install colab-xterm --quiet
!pip install transformers -q
!pip install sentence_transformers -q
!pip install langchain_huggingface --quiet

%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  

In [ ]:
print("\nModule availability:")
modules_to_check = [
    "langchain.chains",
    "langchain.text_splitter",
    "langchain_community.document_loaders",
    "langchain_community.embeddings",
    "langchain_community.vectorstores",
    "langchain_community.llms",
    "langchain_core.output_parsers",
    "langchain.prompts",
    "langchain.schema",
    "langchain_community.output_parsers.rail_parser"
]

for module in modules_to_check:
    try:
        __import__(module)
        print(f"{module}: Available")
    except ImportError as e:
        print(f"{module}: Not available ({str(e)})")


Module availability:
langchain.chains: Available
langchain.text_splitter: Available
langchain_community.document_loaders: Available
langchain_community.embeddings: Available
langchain_community.vectorstores: Available
langchain_community.llms: Available
langchain_core.output_parsers: Available
langchain.prompts: Available
langchain.schema: Available
langchain_community.output_parsers.rail_parser: Available


# Ollama Terminal

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  52035      0 --:--:-- --:--:-- --:--:-- 52077
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install pyngrok -q

In [ ]:
!killall ngrok

ngrok: no process found


In [ ]:
from google.colab import userdata
import os
from pyngrok import ngrok

# replace with actual ngrok authtoken
authtoken = userdata.get('ngrok_key_2')

ngrok.set_auth_token(authtoken)

# Start the Ollama server
def start_ollama_server():
    os.system("ollama serve")

# Create and start a thread to run the Ollama server
import threading
import time

ollama_thread = threading.Thread(target=start_ollama_server)
ollama_thread.start()

time.sleep(10)

# Create a tunnel to the Ollama server
ngrok_tunnel = ngrok.connect(11434)
print(f"ngrok tunnel opened at: {ngrok_tunnel.public_url}")

ngrok tunnel opened at: https://67f9-34-73-209-198.ngrok-free.app


In [ ]:
from pyngrok import ngrok

# List existing tunnels
tunnels = ngrok.get_tunnels()

# Print existing tunnels
print("Existing Tunnels:")
for tunnel in tunnels:
    print(tunnel.public_url)

Existing Tunnels:
https://67f9-34-73-209-198.ngrok-free.app


In [ ]:
import threading

def ollama():
    !ollama serve

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
!ollama pull llama3

Error: listen tcp 127.0.0.1:11434: bind: address already in use
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 529 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  69 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 157 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 210 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   7% ▕▏ 305 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   9% ▕▏ 398 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   9% ▕▏ 437 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...  11% ▕▏ 530 MB/4.7

# Recommendation System


In [ ]:
import pandas as pd
import re
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_community.output_parsers.rail_parser import GuardrailsOutputParser
from typing import List, Dict
import textwrap

class RecommendationSystem:
    def __init__(self, chroma_db_path, llm_model='llama3'):
        self.local_llm = llm_model
        self.loader = CSVLoader(file_path=chroma_db_path)
        self.text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
        self.vectorstore = None
        self.retriever = None
        self.llm = None
        self.prompt_template_retrieval_grader = None
        self.retrieval_grader = None
        self.prompt_template_rag_chain = None
        self.rag_chain = None
        self.prompt_template_hallucination_grader = None
        self.hallucination_grader = None
        self.prompt_template_answer_grader = None
        self.answer_grader = None

        self._initialize_system()

    def _initialize_system(self):
        """Initializes the recommendation system by loading data, setting up vector store, retriever, and LLM."""
        try:
            # Load data
            data = self.loader.load()
            print("Data loaded successfully")

            # Split texts
            texts = self.text_splitter.split_documents(data)
            print(f"Texts split into {len(texts)} chunks")

            # Initialize embeddings and vector store
            self.vectorstore = Chroma.from_documents(
                documents=texts,
                collection_name="rag-chroma",
                embedding=HuggingFaceEmbeddings()
            )
            print("Vector store initialized")

            # Initialize retriever
            self.retriever = self.vectorstore.as_retriever()
            print("Retriever initialized")

        except Exception as e:
            print(f"Error initializing system: {e}")
            self.vectorstore = None
            self.retriever = None

        try:
            self.llm = Ollama(model=self.local_llm, format="json", temperature=0.2)

            self.prompt_template_retrieval_grader = PromptTemplate(
                template="""system You are a grader assessing relevance
                of a retrieved document to a user question. If the document contains keywords related to the user question,
                grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals.
                Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
                Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
                Here is the retrieved document:
                {documents}
                Here is the user question: {question}
                """,
                input_variables=["question", "documents"],
            )

            self.retrieval_grader = self.prompt_template_retrieval_grader | self.llm | JsonOutputParser()

            self.prompt_template_rag_chain = PromptTemplate(
                template="""system: You are Ateema, an AI tour guide. Respond to the user's query using the following format precisely:

                            1. Start with "Hello [user's first name]! I'm Ateema, your AI tour guide."

                            2. Acknowledge the user's destination and group details if mentioned.

                            3. Provide exactly 2 recommendations for each of these categories: Dining, Beverages, Entertainment, Cultural Activities, Outdoor Activities, Educational Activities, and Shopping.

                            4. Format each category and recommendation like below:

                              **Category Name:**
                              1. **Place Name**: Brief description of the place and why it's suitable for the group.
                              2. **Place Name**: Brief description of the place and why it's suitable for the group.

                            5. End with a brief closing statement wishing them a good trip, without inviting further questions.

                            6. End with "I hope you enjoy your trip to Chicago! Have a great time exploring this amazing city."

                            7. Do not include any disclaimers or apologies about the information provided.

                            8. Ensure the entire response is one continuous paragraph without line breaks between sections.

                            9. Bold each recommendation place

                            User question: {question}

                            Context for recommendations: {context}""",
                input_variables=["question", "context"],
            )

            self.rag_chain = self.prompt_template_rag_chain | self.llm | StrOutputParser()

            self.prompt_template_hallucination_grader = PromptTemplate(
                template="""system You are a grader assessing whether an answer is grounded in and supported by a set of facts from the provided documents.
                            Your goal is to ensure the answer does not include any information that contradicts the documents.
                            If the answer contains any information that directly contradicts the documents, grade it as 'no'.
                            If the answer is generally consistent with the facts in the documents and does not introduce significant contradictions, grade it as 'yes'.
                            Missing information is acceptable as long as the provided information aligns with the overall content of the documents.
                            Provide the binary score 'yes' or 'no' as a JSON with a single key 'score' and no preamble or explanation.

                            In addition to that above criteria, evaluate the following aspects:
                            1. **Consistency**: Does the answer generally align with the facts provided in the documents?
                            2. **Completeness**: While the answer may not cover all details, it should not include any incorrect or invented details.
                            3. **Relevance**: The answer should be relevant to the question and broadly based on the facts from the documents.

                            Here are the facts from the documents:
                            {documents}

                            Here is the generated answer:
                            {generation}""",
                input_variables=["generation", "documents"]
            )

            self.hallucination_grader = self.prompt_template_hallucination_grader | self.llm | JsonOutputParser()

            self.prompt_template_answer_grader = PromptTemplate(
                template="""You are a grader assessing whether an answer about a recommended Chicago place is useful and informative.
                            Give a binary score 'yes' or 'no' to indicate whether the answer relates to Chicago.
                            Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
                            Here is the answer:
                            {generation}

                            Here is the question: {question}""",
                input_variables=["generation", "question"],
            )

            self.answer_grader = self.prompt_template_answer_grader | self.llm | JsonOutputParser()

        except Exception as e:
            print(f"Error initializing system: {e}")

    def construct_question(self, customer_info: Dict) -> str:
        """Constructs a question string based on customer information."""
        try:
            first_name = customer_info.get("first_name", "").strip().split()[0]
            travel_city = customer_info.get("travel_city")
            start_date = customer_info.get("start_date")
            end_date = customer_info.get("end_date")
            group_size = customer_info.get("group_size")
            group_age = customer_info.get("group_age")
            pref_dining = customer_info.get("pref_dining")
            pref_beverage = customer_info.get("pref_beverage")
            pref_entertainment = customer_info.get("pref_entertainment")
            pref_cultural = customer_info.get("pref_cultural")
            pref_outdoor = customer_info.get("pref_outdoor")
            pref_education = customer_info.get("pref_education")
            pref_shop = customer_info.get("pref_shop")

            question = (f"My name is {first_name}. I will be visiting {travel_city} from {start_date} to {end_date}. "
                        f"I will be traveling with a group of {group_size} people, aged {group_age}. "
                        f"Our preferences include dining ({pref_dining}), beverages ({pref_beverage}), entertainment ({pref_entertainment}), "
                        f"cultural activities ({pref_cultural}), outdoor activities ({pref_outdoor}), educational activities ({pref_education}), "
                        f"and shopping ({pref_shop}). Can you recommend some interesting places to visit?")

            return question
        except Exception as e:
            print(f"Error constructing question: {e}")
            return ""

    def retrieve(self, state: Dict) -> Dict:
        """Retrieves relevant documents based on the question."""
        try:
            print("---RETRIEVE---")
            question = state["question"]
            documents = self.retriever.invoke(question)
            return {"documents": documents, "question": question}
        except Exception as e:
            print(f"Error retrieving documents: {e}")
            return {}

    def generate(self, state: Dict) -> Dict:
        try:
            print("---GENERATE---")
            question = state["question"]
            documents = state["documents"]
            context = "\n\n".join([doc.page_content for doc in documents])

            prompt = self.prompt_template_rag_chain.format(question=question, context=context)

            generation = ""
            attempts = 0
            max_attempts = 3

            while attempts < max_attempts:
                generation = self.llm.invoke(prompt)
                parsed_generation = StrOutputParser().parse(generation)
                attempts += 1

                print(f"Generated response attempt {attempts}: {parsed_generation}")

                # Check if the response is complete and properly formatted
                if self.is_complete_and_formatted(parsed_generation):
                    return {"documents": documents, "question": question, "generation": parsed_generation.strip()}
                else:
                    print("Incomplete or improperly formatted response, retrying...")

            print(f"Incomplete response after {max_attempts} attempts")
            parsed_generation = "Sorry, I couldn't generate a complete response. Please try again."
            return {"documents": documents, "question": question, "generation": parsed_generation.strip()}  # Added strip() method
        except Exception as e:
            print(f"Error generating response: {e}")
            return {"documents": documents, "question": question, "generation": ""}

    def generate_recommendation(self, customer_info: Dict) -> Dict:
        try:
            question = self.construct_question(customer_info)
            state = {"question": question}

            state = self.route_question(state)
            state = self.retrieve(state)
            state = self.grade_documents(state)
            decision = self.decide_to_generate(state)

            iteration_count = 0
            max_iterations = 3  # Adjusted to allow a reasonable number of iterations

            while decision != "useful" and iteration_count < max_iterations:
                iteration_count += 1
                print(f"Iteration {iteration_count} : Generating response")
                state = self.generate(state)

                # Print the generated response before hallucination check
                print(f"Generated response: {state.get('generation', 'No generation')}")

                decision = self.grade_generation_v_documents_and_question(state)
                print(f"Grading decision: {decision}")

                if decision == "not supported":
                    state = self.retrieve(state)  # Retrieve new documents if the generation is not supported

            final_response = self.parse_response(state.get("generation", ""), state.get("documents", []))
            return final_response
        except Exception as e:
            print(f"Error generating recommendation: {e}")
            return {}


    def is_complete_and_formatted(self, response: str) -> bool:
        """Checks if the generated response is complete and properly formatted."""
        required_sections = [
            "dining",
            "beverages",
            "entertainment",
            "cultural activities",
            "outdoor activities",
            "educational activities",
            "shopping"
        ]

        response_lower = response.lower()

        # Check if all required sections are present
        missing_sections = [section for section in required_sections if section not in response_lower]
        if missing_sections:
            print(f"Format check failed: Missing sections - {', '.join(missing_sections)}")
            return False

        # Check if each section is properly formatted
        section_pattern = r"\*\*(.+?):\*\*\s+1\. \*\*(.+?)\*\*: .+?\s+2\. \*\*(.+?)\*\*: .+?"
        sections = re.findall(section_pattern, response, re.DOTALL)

        if len(sections) != len(required_sections):
            print(f"Format check failed: Expected {len(required_sections)} formatted sections, found {len(sections)}")
            return False

        # Check if all required sections are present in the correct format
        formatted_sections = [section[0].lower() for section in sections]
        missing_formatted_sections = [section for section in required_sections if section not in formatted_sections]
        if missing_formatted_sections:
            print(f"Format check failed: Missing or incorrectly formatted sections - {', '.join(missing_formatted_sections)}")
            return False

        # Check if the response ends with a closing statement
        # if not re.search(r"I hope you enjoy your trip to Chicago! Have a great time exploring this amazing city\.$", response):
        #     print("Format check failed: Missing or incorrect closing statement")
        #     return False

        print("Format check passed: All criteria met")
        return True

    def grade_documents(self, state: Dict) -> Dict:
        """Grades the relevance of retrieved documents to the question."""
        try:
            print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
            question = state["question"]
            documents = state["documents"]
            filtered_docs = []
            for d in documents:
                score = self.retrieval_grader.invoke({"question": question, "documents": d.page_content})
                grade = score['score']
                if grade.lower() == "yes":
                    print("---GRADE: DOCUMENT RELEVANT---")
                    filtered_docs.append(d)
                else:
                    print("---GRADE: DOCUMENT NOT RELEVANT---")
            return {"documents": filtered_docs, "question": question}
        except Exception as e:
            print(f"Error grading documents: {e}")
            return {"documents": [], "question": state["question"]}

    def route_question(self, state: Dict) -> Dict:
        print("---ROUTE QUESTION---")
        question = state["question"]
        print(question)
        return state

    def decide_to_generate(self, state: Dict) -> str:
        """Decides whether to generate a response based on graded documents."""
        print("---ASSESS GRADED DOCUMENTS---")
        filtered_documents = state["documents"]
        if filtered_documents:
            print("---DECISION: GENERATE---")
            return "generate"
        else:
            print("---DECISION: NOT USEFUL---")
            return "not useful"

    def grade_generation_v_documents_and_question(self, state: Dict) -> str:
        print("---CHECK HALLUCINATIONS---")
        documents = state["documents"]
        generation = state["generation"]
        score = self.hallucination_grader.invoke({"documents": documents, "generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
            score = self.answer_grader.invoke({"question": state["question"], "generation": generation})
            grade = score['score']
            if grade == "yes":
                print("---DECISION: GENERATION ADDRESSES QUESTION---")
                return "useful"
            else:
                print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
                return "not useful"
        else:
            print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
            return "not supported"

    def parse_response(self, response: str, documents: List[Document]) -> Dict:
        """Parses the generated response and matches it with the original CSV data to extract URLs."""
        try:
            # Remove any JSON-like wrapping if present
            response = re.sub(r'^.*?"value":\s*"', '', response)
            response = re.sub(r'"}\s*$', '', response)

            # Unescape any escaped characters
            response = response.replace('\\r\\n', '\n').replace('\\"', '"')

            # Extract titles with numbers
            titles_with_numbers = re.findall(r'\d+\.\s*\*\*(.*?)\*\*:', response, re.DOTALL)

            normalized_titles = [self._normalize_text(title) for title in titles_with_numbers]

            # Load the original CSV file to match against the title_cleaned and get the image URLs
            df = pd.read_csv(self.loader.file_path)
            df['normalized_title_cleaned'] = df['title_cleaned'].apply(self._normalize_text)

            recommendations = []
            url_info = []
            for index, (title_with_number, normalized_title) in enumerate(zip(titles_with_numbers, normalized_titles), start=1):
                match = df[df['normalized_title_cleaned'].str.contains(normalized_title, na=False, case=False)]
                print(f"Matching '{normalized_title}' with titles in CSV, found matches:", match)

                if not match.empty:
                    url = match.iloc[0]['image']
                    match_status = "Yes"
                    url_status = url if pd.notna(url) else "No URL available"
                else:
                    url_status = "No URL available"
                    match_status = "No"
                recommendations.append(f"{index}. {title_with_number.strip()}")
                url_info.append({"Title": title_with_number.strip(), "Match": match_status, "URL": url_status})

            # Create a human-readable formatted response
            formatted_prompt = response.strip()

            formatted_response = {
                "recommendation": formatted_prompt,
                "url_information": url_info
            }

            return formatted_response
        except Exception as e:
            print(f"Error parsing response: {e}")
            return {"recommendation": response, "url_information": []}

    def _normalize_text(self, text: str) -> str:
        """Normalizes text for matching purposes."""
        return re.sub(r'\s+', '', text.lower().replace("the", "").replace("'", "").replace("-", "").replace(",", "").replace(".", ""))

# Recommendation.py

In [ ]:
!python Recommendation_System.py

from Recommendation_System import RecommendationSystem
import json

In [ ]:
import inspect

print(inspect.getmembers(RecommendationSystem))

[('__class__', <class 'type'>), ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>), ('__dict__', mappingproxy({'__module__': 'Recommendation_System', '__init__': <function RecommendationSystem.__init__ at 0x7a5e42a44d30>, '_initialize_system': <function RecommendationSystem._initialize_system at 0x7a5e42a464d0>, 'construct_question': <function RecommendationSystem.construct_question at 0x7a5e42a46710>, 'retrieve': <function RecommendationSystem.retrieve at 0x7a5e42a467a0>, 'generate': <function RecommendationSystem.generate at 0x7a5e42a46830>, 'generate_recommendation': <function RecommendationSystem.generate_recommendation at 0x7a5e42a468c0>, 'is_complete_and_formatted': <function RecommendationSystem.is_complete_and_formatted at 0x7a5e42a46950>, 'grade_documents': <function RecommendationSystem.grade_documents at 0x7a5e42a469e0>, 'route_question': <function RecommendationSystem.route_question at 0x7a5e42a46a70>, 'decide_to_generate': <function RecommendationSystem.deci

In [ ]:
for name, method in inspect.getmembers(RecommendationSystem, inspect.isfunction):
    print(f"Method: {name}")

Method: __init__
Method: _initialize_system
Method: _normalize_text
Method: construct_question
Method: decide_to_generate
Method: generate
Method: generate_recommendation
Method: grade_documents
Method: grade_generation_v_documents_and_question
Method: is_complete_and_formatted
Method: parse_response
Method: retrieve
Method: route_question


# survey Info

In [ ]:
!pip install pymongo --quiet
!pip install flask pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 23.9 MB/s eta 0:00:00


In [ ]:
chroma_db_path = "data_main.csv"
recommendation_system = RecommendationSystem(chroma_db_path)
# recommendation_system.initialize_system()

Data loaded successfully
Texts split into 813 chunks


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store initialized
Retriever initialized


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from datetime import datetime
from google.colab import userdata
from pyngrok import ngrok
import json


def format_text(text):
    paragraphs = text.strip().split('\r\n\r\n')
    formatted_paragraphs = []
    for para in paragraphs:
        para = textwrap.dedent(para)
        para = textwrap.fill(para, width=80)
        formatted_paragraphs.append(para)
    return '\n\n'.join(formatted_paragraphs)

app = Flask(__name__)

@app.route('/webhook', methods=['POST'])
def typeform_webhook():
    try:
        request_data = request.get_json()
        if request_data is None:
            return jsonify({'message': 'No JSON data received'}), 400

        # Now request_data is already a dictionary, no need to use json.loads()
        survey_data = request_data
        df = pd.DataFrame([survey_data])
        customer_info = df.iloc[0].to_dict()
        response = recommendation_system.generate_recommendation(customer_info)

        # Prepare the data to send
        data_to_send = {
            'recommendation': response["recommendation"],
            'url_information': response["url_information"]
        }

        # Print the data to send
        print("\nData to Send:")
        print(json.dumps(data_to_send, indent=2))

        response = {
            "message": "Confirm received data",
            "recommendation": data_to_send
        }

        return jsonify(response), 200
    except Exception as e:
        print(f"Error processing request: {str(e)}")
        return jsonify({'message': 'Error processing request', 'error': str(e)}), 500


if __name__ == '__main__':
    ngrok.set_auth_token(userdata.get('ngrok_key'))
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    app.run(port=5000)

Public URL: https://2dff-34-73-209-198.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


---ROUTE QUESTION---
My name is Lorem. I will be visiting Chicago from 2017-02-06 to 2017-02-06. I will be traveling with a group of 42 people, aged 20 -. Our preferences include dining (Gourmet, Chicago, Outdoor, Family-Friendly, Food), beverages (Wine, Cocktails, Speakeasies, Bars, Rooftop, Craft, Distilleries, Sports, Irish, Waterfront), entertainment (Live, Clubs, Concerts, Festivals, Theater, Comedy), cultural activities (Art, Immersive, Sporting, Attractions, Neighborhoods, Architecture, History, Street), outdoor activities (Outdoor, Boat, Rentals:, Lakefront, River, Walking), educational activities (Kid-Friendly, Art, Landmarks, Cooking, Haunted), and shopping (Shopping, Local, Farmers). Can you recommend some interesting places to visit?
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
Iterati

INFO:werkzeug:127.0.0.1 - - [11/Aug/2024 22:47:01] "POST /webhook HTTP/1.1" 200 -


---DECISION: GENERATION ADDRESSES QUESTION---
Grading decision: useful
Matching 'alinea' with titles in CSV, found matches:       title                                        description  \
567  Alinea  ThereÛªs probably no greater star chef practi...   
577  Alinea  ItÛªs hard to think of an honor that  hasnÛª...   
589  Alinea  Enjoy a unique, multi-course tasting menu expe...   

                                                 image         Source  \
567  https://cdn.choosechicago.com/uploads/2019/08/...  choosechicago   
577  https://cdn.choosechicago.com/uploads/2019/06/...  choosechicago   
589                                                NaN  choosechicago   

                        category  \
567             luxury-top chefs   
577           luxury-fine dining   
589  luxury-three stars michelin   

                                          text_cleaned title_cleaned  \
567  probably greater star chef practicing modernis...        alinea   
577  hard think honor three m

In [ ]:
import json
import textwrap

def format_text(text):
    paragraphs = text.strip().split('\r\n\r\n')
    formatted_paragraphs = []
    for para in paragraphs:
        para = textwrap.dedent(para)
        para = textwrap.fill(para, width=80)
        formatted_paragraphs.append(para)
    return '\n\n'.join(formatted_paragraphs)

def recommendation():
    # Convert the first row to a dictionary
    customer_info = customer_data_df.iloc[0].to_dict()

    # Generate prompt
    response = recommendation_system.generate_recommendation(customer_info)

    # Format the recommendation
    formatted_recommendation = format_text(response["recommendation"])

    # Print the formatted recommendation
    print("Recommendation:")
    print(formatted_recommendation)

    # Print the URL information
    print("\nURL Information:")
    print(json.dumps(response["url_information"], indent=2))

    # Prepare the data to send
    data_to_send = {
        'recommendation': formatted_recommendation,
        'url_information': response["url_information"]
    }

    # Print the data to send
    print("\nData to Send:")
    print(json.dumps(data_to_send, indent=2))

    # Save the result to a text file
    with open('recommendation_result.txt', 'w') as file:
        file.write("Recommendation:\n")
        file.write(formatted_recommendation)
        file.write("\n\nURL Information:\n")
        file.write(json.dumps(response["url_information"], indent=2))

    return data_to_send

if __name__ == "__main__":
    data_to_send = recommendation()

---ROUTE QUESTION---
My name is Utku. I will be visiting Chicago from 2024-12-01 to 2024-12-10. I will be traveling with a group of 1 people, aged 31-40. Our preferences include dining (Gourmet, Chicago), beverages (Wine, Cocktails), entertainment (Live, Clubs), cultural activities (Art, Immersive), outdoor activities (Outdoor, Boat), educational activities (Kid-Friendly, Art), and shopping (Shopping). Can you recommend some interesting places to visit?
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
Iteration 1 : Generating response
---GENERATE---


ERROR:pyngrok.process.ngrok:t=2024-07-16T16:34:38+0000 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=d365aabac4c4 clientid=2dbfe2b1945ef850eb3d8f9f83aee4de


In [ ]:
# import requests
# import pandas as pd

# # Replace with the actual public URL provided by ngrok
# public_url = "https://0fd4-34-106-245-186.ngrok-free.app"

# response = requests.get(f"{public_url}/get_latest_data")

# if response.status_code == 200:
#     data = response.json()
#     if isinstance(data, list) and len(data) > 0:
#         customer_data_df = pd.DataFrame(data)
#         print("Latest DataFrame retrieved from the server:")
#     else:
#         print("No data available.")
# else:
#     print(f"Failed to retrieve data. Status code: {response.status_code}")
#     print(response.text)

# customer_data_df.head()

Latest DataFrame retrieved from the server:


first_name last_name  phone_number                email travel_city  \
0      Cindy     Zhang  +19492360675  cnkung112@gmail.com     Chicago   

   start_date    end_date   group_type  group_size group_age  \
0  2024-12-01  2024-12-10  Girl's trip           4     21-30   

        pref_dining                          pref_beverage pref_entertainment  \
0  Gourmet, Chicago  Wine, Cocktails, Speakeasies, Rooftop     Clubs, Theater   

                                       pref_cultural            pref_outdoor  \
0  Attractions, Art, Architecture, History, Neigh...  Lakefront, Boat, River   

   pref_education pref_shop                                 comment  \
0  Art, Landmarks  Shopping  I want to drive to further attraction.   

  video_length      timestamp  
0   30 seconds  1721007474163

In [ ]:
import pandas as pd
import json

data = {
    "first_name": ["Utku"],
    "last_name": ["Pamuksuz"],
    "phone_number": ["+19492360674"],
    "email": ["amuksuz@uchicago.edu"],
    "travel_city": ["Chicago"],
    # update to current or future date
    "start_date": ["2024-12-01"],
    "end_date": ["2024-12-10"],
    "group_type": ["Solo Trip"],
    "group_size": [1],
    "group_age": ["31-40"],
    "pref_dining": ["Gourmet, Chicago"],
    "pref_beverage": ["Wine, Cocktails"],
    "pref_entertainment": ["Live, Clubs"],
    "pref_cultural": ["Art, Immersive"],
    "pref_outdoor": ["Outdoor, Boat"],
    "pref_education": ["Kid-Friendly, Art"],
    "pref_shop": ["Shopping"],
    "comment": ["I want to visit local bar and sporting event"],
    "video_length": ["30 seconds"],
    "timestamp":["1719366581158"]
}

customer_data_df = pd.DataFrame(data)
customer_data_df.head()

first_name last_name  phone_number                 email travel_city  \
0       Utku  Pamuksuz  +19492360674  amuksuz@uchicago.edu     Chicago   

   start_date    end_date group_type  group_size group_age       pref_dining  \
0  2024-12-01  2024-12-10  Solo Trip           1     31-40  Gourmet, Chicago   

     pref_beverage pref_entertainment   pref_cultural   pref_outdoor  \
0  Wine, Cocktails        Live, Clubs  Art, Immersive  Outdoor, Boat   

      pref_education pref_shop                                       comment  \
0  Kid-Friendly, Art  Shopping  I want to visit local bar and sporting event   

  video_length      timestamp  
0   30 seconds  1719366581158

# Run

In [ ]:
# if __name__ == "__main__":
#     data_to_send = recommendation()

# app = Flask(__name__)

# # Initialize the recommendation system with the database path
# chroma_db_path = "/Users/hyojun/Desktop/data/data_main.csv"
# recommendation_system = RecommendationSystem(chroma_db_path)

# @app.route('/recommend', methods=['POST'])
# def recommend():
#     survey_data = request.get_json()
#     itinerary = recommendation_system.generate_prompt(survey_data)
#     return jsonify(itinerary)

# if __name__ == "__main__":
#     app.run()


**Archive**

In [ ]:
# def parse_response(self, response: str, documents: List[Document]) -> Dict:
#         """Parses the generated response and matches it with the original CSV data to extract URLs."""
#         try:
#             # Remove any JSON-like wrapping if present
#             response = re.sub(r'^.*?"value":\s*?"', '', response)
#             response = re.sub(r'"}\s*$', '', response)

#             # Unescape any escaped characters
#             response = response.replace('\\r\\n', '\n').replace('\\"', '"')

#             # Extract titles with numbers
#             titles_with_numbers = re.findall(r'\d+\.\s*\*\*(.*?)\*\*:', response, re.DOTALL)

#             normalized_titles = [self._normalize_text(title) for title in titles_with_numbers]

#             # Load the original CSV file to match against the title_cleaned and get the image URLs
#             df = pd.read_csv(self.loader.file_path)
#             df['normalized_title_cleaned'] = df['title_cleaned'].apply(self._normalize_text)

#             recommendations = []
#             url_info = []
#             for index, (title_with_number, normalized_title) in enumerate(zip(titles_with_numbers, normalized_titles), start=1):
#                 match = df[df['normalized_title_cleaned'].str.contains(normalized_title, na=False, case=False)]
#                 print(f"Matching '{normalized_title}' with titles in CSV, found matches:", match)

#                 if not match.empty:
#                     url = match.iloc[0]['image']
#                     match_status = "Yes"
#                     url_status = url if pd.notna(url) else "No URL available"
#                 else:
#                     url_status = "No URL available"
#                     match_status = "No"
#                 recommendations.append(f"{index}. {title_with_number.strip()}")
#                 url_info.append({"Title": title_with_number.strip(), "Match": match_status, "URL": url_status})

#             # Create a human-readable formatted response
#             formatted_prompt = response.strip()

#             formatted_response = {
#                 "recommendation": formatted_prompt,
#                 "url_information": url_info
#             }

#             return formatted_response
#         except Exception as e:
#             print(f"Error parsing response: {e}")
#             return {"recommendation": response, "url_information": []}

In [ ]:
# import subprocess

# def run_command(command):
#     try:
#         result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#         print(result.stdout)
#         return result
#     except subprocess.CalledProcessError as e:
#         print(f"Error running command: {command}")
#         print(e.stdout)
#         print(e.stderr)
#         raise

# # Install the ollama tool
# run_command("curl -fsSL https://ollama.com/install.sh | sh")

# # Start the ollama service in the background
# subprocess.Popen("ollama serve &", shell=True)

# # Verify that the ollama service is running
# import time
# time.sleep(5)  # Give some time for the service to start

# # Pull the llama3 model
# run_command("ollama pull llama3")

In [ ]:
# import pandas as pd
# survey_data_path = "latest_data.csv"
# customer_data_df = pd.read_csv(survey_data_path)
# customer_data_df.head()